In [ ]:
def categorize_label(lbl, ncls):
  cat = to_categorical(lbl, num_classes = ncls)
  return cat

In [ ]:
import cv2
from tensorflow.keras.utils import to_categorical

def get_labeled_image(image, label, is_categorical=False):
    if not is_categorical:
        label = categorize_label(label, num_classes=4).astype(np.uint8)

    image = cv2.normalize(image[:, :, :, 0], None, alpha=0, beta=255,
                          norm_type=cv2.NORM_MINMAX, dtype=cv2.CV_32F).astype(
        np.uint8)

    labeled_image = np.zeros_like(label[:, :, :, 1:])

    # remove tumor part from image
    labeled_image[:, :, :, 0] = image * (label[:, :, :, 0])
    labeled_image[:, :, :, 1] = image * (label[:, :, :, 0])
    labeled_image[:, :, :, 2] = image * (label[:, :, :, 0])


    # color labels
    labeled_image += label[:, :, :, 1:] * 255
    return labeled_image


In [ ]:
import imageio
from IPython.display import Image

def visualize_data_gif(data_):
    images = []
    for i in range(data_.shape[0]):
        x = data_[min(i, data_.shape[0] - 1), :, :]
        y = data_[:, min(i, data_.shape[1] - 1), :]
        z = data_[:, :, min(i, data_.shape[2] - 1)]
        img = np.concatenate((x, y, z), axis=1)
        images.append(img)
    imageio.mimsave("/content/drive/MyDrive/training/gf.gif", images, duration=0.03)
    return Image(filename="/content/drive/MyDrive/training/gf.gif")

In [ ]:
import matplotlib.pyplot as plt
%matplotlib inline
def plot_image_grid(image):
    data_all = []

    data_all.append(image)

    fig, ax = plt.subplots(3, 6, figsize=[16, 9])

    # coronal plane
    coronal = np.transpose(data_all, [1, 3, 2, 4, 0])
    coronal = np.rot90(coronal, 1)

    # transversal plane
    transversal = np.transpose(data_all, [2, 1, 3, 4, 0])
    transversal = np.rot90(transversal, 2)

    # sagittal plane
    sagittal = np.transpose(data_all, [2, 3, 1, 4, 0])
    sagittal = np.rot90(sagittal, 1)

    for i in range(6):
        n = np.random.randint(coronal.shape[2])
        ax[0][i].imshow(np.squeeze(coronal[:, :, n, :]))
        ax[0][i].set_xticks([])
        ax[0][i].set_yticks([])
        if i == 0:
            ax[0][i].set_ylabel('Coronal', fontsize=15)

    for i in range(6):
        n = np.random.randint(transversal.shape[2])
        ax[1][i].imshow(np.squeeze(transversal[:, :, n, :]))
        ax[1][i].set_xticks([])
        ax[1][i].set_yticks([])
        if i == 0:
            ax[1][i].set_ylabel('Transversal', fontsize=15)

    for i in range(6):
        n = np.random.randint(sagittal.shape[2])
        ax[2][i].imshow(np.squeeze(sagittal[:, :, n, :]))
        ax[2][i].set_xticks([])
        ax[2][i].set_yticks([])
        if i == 0:
            ax[2][i].set_ylabel('Sagittal', fontsize=15)

    fig.subplots_adjust(wspace=0, hspace=0)

In [ ]:
import numpy as np
import nibabel as nib
import os
import tensorflow as tf

def load_case(image_nifty_file, label_nifty_file):
    # load the image and label file, get the image content and return a numpy array for each
    image = np.array(nib.load(image_nifty_file).get_fdata())
    label = np.array(nib.load(label_nifty_file).get_fdata())
    
    return image, label

In [ ]:
def show_slices(slices):
   
   fig, axes = plt.subplots(1, len(slices))
   for i, slice in enumerate(slices):
       axes[i].imshow(slice.T, cmap="gray", origin="lower")


In [ ]:
def suit(image, window_center, window_width):
  image_min = window_center - window_width//2
  image_max = window_center + window_width//2
  image[image<image_min] = image_min
  image[image>image_max] = image_max
 
  return image

In [ ]:
d = all_data[1].copy()
ct = suit(d, -600, 1000)
slice_0 = ct[190, :, :]
slice_1 = ct[:, 157, :]
slice_2 = ct[:, :, 150]
show_slices([slice_0, slice_1, slice_2])

In [ ]:
def intensity_bin(np_array, min, max):
  clipped = np_array.clip(min, max)
  clipped[clipped != max] = 1
  clipped[clipped == max] = 0
  return clipped

In [ ]:
slice_0 = ct[0, :, :]
slice_1 = ct[:, 0, :]
slice_2 = ct[:, :, 100]
show_slices([slice_0, slice_2])

In [ ]:
a = intensity_bin(all_data[8][:,:,150],0, 1000)
plt.imshow(a)

In [ ]:
def show_contour(image, contours):
  fig, ax = plt.subplots()
  ax.imshow(image.T, cmap = plt.cm.gray)
  for contour in contours:
    ax.plot(contour[:, 0], contour[:, 1], linewidth = 1)
    plt.show()

In [ ]:
import skimage.transform as skTrans

def resize_nii(img):
  rsd = skTrans.resize(img, (256,256,256,), order=1, preserve_range = True)
  return rsd

In [ ]:
rimg = resize_nii(trial)
rmsk = resize_nii(name)

In [ ]:
def new_data(images, masks):
  for image, mask in images, masks:
    i = nib.save(image, )
    m = nib.save(mask,  )
    return i, m

In [ ]:
pip install patchify

In [ ]:
from patchify import patchify

pth_img = patchify(rimg, (64,64,64), step=64)
pth_mask = patchify(rmsk, (64,64,64), step=64)

input_img = np.reshape(pth_img, (-1, pth_img.shape[3], pth_img.shape[4], pth_img.shape[5]))
input_mask = np.reshape(pth_mask, (-1, pth_mask.shape[3], pth_mask.shape[4], pth_mask.shape[5]))

In [ ]:
train_img = np.stack((input_img,)*3, axis=-1)
train_mask = np.expand_dims(input_mask, axis = 4)

(64, 64, 64, 64, 1)

In [ ]:
train_img_mask = categorize_label(train_mask, ncls=4)

In [ ]:
train_img_mask.shape

(64, 64, 64, 64, 4)

**Model** 

In [ ]:
import keras
from keras import backend as K
from keras.models import Model
from keras.layers import Input, Conv3D, MaxPooling3D, UpSampling3D, Activation, BatchNormalization, Conv3DTranspose
from tensorflow.keras.optimizers import Adam
from keras.layers import Concatenate

In [ ]:
def jaccard_coeff(y_true, y_pred):
  y_true_f = K.flatten(y_true)
  y_pred_f = K.flatten(y_pred)
  intersection = K.sum(y_true_f * y_pred_f)
  return (intersection + 1.0) / (K.sum(y_true_f) + K.sum(y_pred_f) - intersection + 1.0)

In [ ]:
def jaccard_coeff_loss(y_true, y_pred):
  return -jaccard_coeff(y_true, y_pred)

In [ ]:
input_layer = Input(shape=(64, 64, 64, 3))

In [ ]:
def bn_conv_block(input, num_filters):
  x = Conv3D(num_filters, 
             kernel_size=(3,3,3),
             padding='same',
             strides=(1,1,1)
              )(input_layer)  
  x = BatchNormalization()(x)
  x = Activation('relu')(x) 

  x =  Conv3D(num_filters, 
             kernel_size=(3,3,3),
             padding='same',
             strides=(1,1,1)
              )(x)
  x = BatchNormalization()(x)
  x = Activation('relu')(x)

  return x

In [ ]:
def dropout_conv_block(input, num_filters):
  x = Conv3D(num_filters, 
             kernel_size=(3,3,3),
             padding='same',
             strides=(1,1,1)
              )(input_layer)  
  x = dropout(x)
  x = Activation('relu')(x) 

  x =  Conv3D(num_filters, 
             kernel_size=(3,3,3),
             padding='same',
             strides=(1,1,1)
              )(x)
  x = dropout(x)
  x = Activation('relu')(x)

  return x

In [ ]:
def encoder_block(input, num_filters):
  x = bn_conv_block(input, num_filters)
  p = MaxPooling3D(pool_size = (2,2,2))(x)
  return x,p

In [ ]:
def transpose_decoder_block(input,skip_features, num_filters):
  x = Conv3DTranspose(num_filters, kernel_size=(2,2,2), strides=1, padding='same')(input)
  x = Concatenate()([x, skip_features])
  x = bn_conv_block(x, num_filters)
  
  return x

In [ ]:
def Unet3D(input):

  c1, p1 = encoder_block(input, 64)
  c2, p2 = encoder_block(p1, 128)
  c3, p3 = encoder_block(p2, 256)
  c4, p4 = encoder_block(p3, 512)

  b1 = bn_conv_block(p4, 1024)
  
  d1 = transpose_decoder_block(b1, c4 , 512)
  d2 = transpose_decoder_block(d1, c3, 256)
  d3 = transpose_decoder_block(d2, c2, 128)
  d4 = transpose_decoder_block(d3, c1, 64)

  output = Conv3D(4, 1, padding='same', activation='sigmoid')(d4)

  model = Model(input, output, name="U-Net")
  return model

In [ ]:
optimizer = Adam
model = Unet3D(input_layer)
model.compile(optimizer=optimizer(learning_rate=0.00001),
              loss='binary_crossentropy',
              metrics=['categorical_accuracy']
             )
#model.compile(optimizer=Adam(lr=0.00001),loss='jaccard_coeff_loss',metrics=['jaccard_coeff'])


In [ ]:
print(model.summary())

Model: "U-Net"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_4 (InputLayer)         [(None, 64, 64, 64, 3)]   0         
_________________________________________________________________
conv3d_54 (Conv3D)           (None, 64, 64, 64, 64)    5248      
_________________________________________________________________
batch_normalization_52 (Batc (None, 64, 64, 64, 64)    256       
_________________________________________________________________
activation_52 (Activation)   (None, 64, 64, 64, 64)    0         
_________________________________________________________________
conv3d_55 (Conv3D)           (None, 64, 64, 64, 64)    110656    
_________________________________________________________________
batch_normalization_53 (Batc (None, 64, 64, 64, 64)    256       
_________________________________________________________________
activation_53 (Activation)   (None, 64, 64, 64, 64)    0     

In [ ]:
model.fit(train_img, train_img_mask, epochs = 5, steps_per_epoch = 5)

Epoch 1/5
5/5 [==============================] - 355s 70s/step - loss: 0.9274 - categorical_accuracy: 0.0148
Epoch 2/5
5/5 [==============================] - 353s 71s/step - loss: 0.9018 - categorical_accuracy: 0.0145
Epoch 3/5
5/5 [==============================] - 350s 70s/step - loss: 0.8769 - categorical_accuracy: 0.0148
Epoch 4/5
5/5 [==============================] - 358s 72s/step - loss: 0.8554 - categorical_accuracy: 0.0195
Epoch 5/5
5/5 [==============================] - 349s 69s/step - loss: 0.8396 - categorical_accuracy: 0.0364
